In [ ]:
import keras
import tensorflow as tf
import numpy as np
from keras import layers
from keras import ops
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# load data
import numpy as np
x = np.load("/content/drive/My Drive/x_letters.npy")
y=np.load("/content/drive/My Drive/y_letters.npy")

In [ ]:
x.shape, y.shape

((88799, 28, 28), (88799,))

In [ ]:
y = to_categorical(y, num_classes=26)

X_train_full, X_test, Y_train_full, Y_test = train_test_split(x, y, stratify = y
                                                              )
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train_full, Y_train_full, stratify = Y_train_full)
X_train, X_valid, X_test = X_train/255., X_valid/255., X_test/255.

print(f"x_train shape: {X_train.shape}")
print(f"x_valid shape: {X_valid.shape}")
print(f"x_test shape: {X_test.shape}")

x_train shape: (49949, 28, 28)
x_valid shape: (16650, 28, 28)
x_test shape: (22200, 28, 28)


In [ ]:
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.seed_generator = keras.random.SeedGenerator(1337)

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, data):
        # Sample random points in the latent space

        real_images, real_classes = data
        batch_size = ops.shape(real_images)[0]
        random_latent_vectors = keras.random.normal(
            shape=(batch_size, self.latent_dim),
            seed = self.seed_generator
        )

        # Decode them to fake images
        generated_images = self.generator([random_latent_vectors, real_classes])

        # Combine them with real images
        real_images = real_images * 2 - 1  # Rescale to [-1, 1], because the generator outputs images in the same range (using tanh as last activation function)
        combined_images = ops.concatenate([generated_images, real_images], axis=0)
        combined_classes = ops.concatenate([real_classes, real_classes], axis=0)

        # Assemble labels discriminating real from fake images
        labels = ops.concatenate(
            [ops.ones((batch_size, 1)) * 0.9, ops.zeros((batch_size, 1))], axis=0
        )
        # Add random noise to the labels - important trick!
        #basically makes sure the discriminator is not perfect,
        #so that the generator never has a chance to learn
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator([combined_images, combined_classes])
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)

        # Clip gradients for training stability
        # basically makes sure the gradients are not too big
        # by clipping them to a certain value if they are over that value
        grads = [tf.clip_by_value(g, -1.0, 1.0) for g in grads]

        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space
        random_latent_vectors = keras.random.normal(
            shape=(batch_size, self.latent_dim),
            seed = self.seed_generator
        )

        # Assemble labels that say "all real images"
        misleading_labels = ops.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            generated_images = self.generator([random_latent_vectors, real_classes])
            predictions = self.discriminator([generated_images, real_classes])
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        # Clip gradients for stability
        grads = [tf.clip_by_value(g, -1.0, 1.0) for g in grads]
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Update metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }